In [1]:
from lxml import etree
from pathlib import Path
from pymongo import MongoClient

In [2]:
db=MongoClient()

In [3]:
stations=db.stations_non_germany

In [4]:
stations.create_collection("logs")
logs_doc=stations.logs

In [5]:
sorted(list(stations.list_collections()),key=lambda station:station['name'])

[{'name': 'logs',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('92b10fe0-51fd-48e6-a808-d1f1ed6157d4')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [6]:
def get_safe_first(values):
    if values:
        return values[0]
    else:
        return ""

In [7]:
path = Path("./Archivdaten_2019")

In [8]:
files=[file for file in path.iterdir() if not file.is_dir()]

In [9]:
for file in files:
    if file.stem not in stations.list_collection_names():
        stations.create_collection(file.stem)
        print(f"collection {file.stem} created")

collection 2019_3SAT created
collection 2019_ARD created
collection 2019_ARTE created
collection 2019_BFS created
collection 2019_DMAX created
collection 2019_DW-tv created
collection 2019_EinsFestival created
collection 2019_HF created
collection 2019_K1 created
collection 2019_K1DOKU created
collection 2019_KKA created
collection 2019_MDR created
collection 2019_N24DOKU created
collection 2019_N3_Hamburg created
collection 2019_N3_Radio_Bremen created
collection 2019_NTV created
collection 2019_PHOENIX created
collection 2019_PRO7 created
collection 2019_ProSiebenFun created
collection 2019_ProSieben_MAXX created
collection 2019_rbb_Berlin created
collection 2019_RTL created
collection 2019_RTL2 created
collection 2019_RTLplus created
collection 2019_RTLS created
collection 2019_RTL_Nitro created
collection 2019_SAT1 created
collection 2019_Sat1Gold created
collection 2019_sixx created
collection 2019_Sport1 created
collection 2019_SWR_BW created
collection 2019_SWR_SR created
collec

In [10]:
parser = etree.XMLParser(recover=True)

In [11]:
def save_show(show,station,extras={}):
    params={
            'show_id':show.xpath("./sendung//stammdaten//metadaten/id/text()")[0],
            'title':show.xpath("./sendung//stammdaten//programm/titel/text()")[0],
    }
    values={**params,**extras}
    values['subtitle']=get_safe_first(show.xpath("./sendung//stammdaten//programm/nebentitel/text()")) 
    values['originaltitel']=get_safe_first(show.xpath("./sendung//stammdaten//programm/originaltitel/text()")) 
    values['sender_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/sender_genre/text()"))
    values['dmb_haupt_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/dmb_genre/dmb_haupt_genre/text()"))
    values['dmb_neben_genre']=get_safe_first(show.xpath("./sendung//stammdaten//programm/dmb_genre/dmb_neben_genre/text()"))
    values['produktion_land']=get_safe_first(show.xpath("./sendung//stammdaten//programm/produktion/land/text()"))
    values['produktion_jahr']=get_safe_first(show.xpath("./sendung//stammdaten//programm/produktion/jahr/text()"))
    values['hinweissender']=get_safe_first(show.xpath("./sendung//stammdaten//programm/hinweissender/text()"))
    values['titel_ergaenzung']=get_safe_first(show.xpath("./sendung//stammdaten//programm/titel_ergaenzung/text()"))
    values['hinweissender']=get_safe_first(show.xpath("./sendung//stammdaten//programm/hinweissender/text()"))
    values['ausstrahlung_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/start/text()"))
    values['ausstrahlung_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/wh-text/text()"))
    values['ausstrahlung_sender_start']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/sender_start/text()"))
    values['ausstrahlung_stop']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/stop/text()"))
    values['ausstrahlung_dauer']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/dauer/text()"))
    values['nettolaenge']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/nettolaenge/text()"))
    values['ausstrahlung_sender']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/sender/text()"))
    values['produzierender_sender']=get_safe_first(show.xpath("./sendung//stammdaten//ausstrahlung/produzierender_sender/text()"))
    values['dmbInfoset_ProduktionsLaender_ProduktionsLand']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/ProduktionsLaender/ProduktionsLand/text()"))
    values['dmbFolgetitel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbFolgetitel/text()"))
    values['dmbStaffel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbStaffel/text()"))
    values['dmbFolge']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbFolge/text()"))
    values['dmbProduktionsJahr']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbProduktionsJahr/text()"))
    values['dmbOriginalFolgetitel']=get_safe_first(show.xpath("./sendung//stammdaten//dmbInfoset/dmbOriginalFolgetitel/text()"))
    values['staff']=show.xpath("./sendung//stammdaten//dmbInfoset/StaffList/Staff/Person/Fullname/text()")
    produktionsLand_nodes=show.xpath("./sendung//stammdaten//dmbInfoset/ProduktionsLaender/ProduktionsLand")
    produktionsLand=[{"land":node.text,'kfz':node.attrib.get("dmbstd:kfz"),'alpha3':node.attrib.get("dmbstd:alpha3") } for node in produktionsLand_nodes]
    values['produktionsLand']=produktionsLand
    staff_nodes=show.xpath("./sendung//stammdaten//dmbInfoset/StaffList/Staff")
    staff=[{"staff_id":node.xpath("./Person")[0].attrib.get("dmbstd:id"),"staff_function":node.attrib.get("dmbstd:Function"),"fullname":node.xpath("./Person/Fullname")[0].text} for node in staff_nodes]
    values['staff']=staff
    values['crew_united_url']=""
    values['crew_united_productions']=[]
    values['crew_united_koproduzent_productions']=[]
    values['cu_first_showing_country']=""
    values['cu_first_showing_station']=""
    values['cu_first_showing_date_time']=""
    values['imdb_id']=""
    values['imdb_link']=""
    values['imdb_distributors']=[]
    values['imdb_productions']=[]
    values['imdb_flag']=""
    values['cu_flag']=""
    
    
    
    station.replace_one(params,values,upsert=True)

In [12]:
for file in files:
    if file.stem not in stations.list_collection_names():
        stations.create_collection(file.stem)
    station=stations[file.stem]
    tree = etree.parse(str(file),parser=parser)
    shows=tree.xpath(".//daten/einfuegen")
    for show in shows:
        save_show(show,station)
#     empty_shows=[]
#     shows=tree.xpath(".//daten/einfuegen")
#     for show in shows:
#         production_land=show.xpath(".//stammdaten//produktion/land/text()")
#         production_land_1=show.xpath(".//stammdaten//dmbInfoset//dmbProduktionsLand/text()")
#         production_land_2=show.xpath(".//stammdaten//dmbInfoset//ProduktionsLaender/ProduktionsLand/text()")
#         if all([not production_land,not production_land_1,not production_land_2]) :
#             empty_shows.append(show)
#     for show in empty_shows:
#         save_show(show,empties,{"station":file.stem})
    print(f"{file.stem} total shows {len(shows)}")

2019_3SAT total shows 12532
2019_ARD total shows 13280
2019_ARTE total shows 11870
2019_BFS total shows 13371
2019_DMAX total shows 11118
2019_DW-tv total shows 24774
2019_EinsFestival total shows 11122
2019_HF total shows 13431
2019_K1 total shows 9142
2019_K1DOKU total shows 11842
2019_KKA total shows 23298
2019_MDR total shows 15514
2019_N24DOKU total shows 10481
2019_N3_Hamburg total shows 13667
2019_N3_Radio_Bremen total shows 14520
2019_NTV total shows 21993
2019_PHOENIX total shows 11517
2019_PRO7 total shows 14616
2019_ProSiebenFun total shows 16838
2019_ProSieben_MAXX total shows 15903
2019_rbb_Berlin total shows 13876
2019_RTL total shows 10026
2019_RTL2 total shows 7434
2019_RTLplus total shows 11860
2019_RTLS total shows 13940
2019_RTL_Nitro total shows 11732
2019_SAT1 total shows 8265
2019_Sat1Gold total shows 12422
2019_sixx total shows 11104
2019_Sport1 total shows 15177
2019_SWR_BW total shows 14432
2019_SWR_SR total shows 14447
2019_tagesschau24 total shows 14323
2019_

In [13]:
de_stations=db.stations

In [38]:
station_names=[station['name'] for station in
                        sorted(list(stations.list_collections()), key=lambda station: station['name']) if
                        station['name'] not in ['logs', 'empties']]

In [39]:
station_names

['2019_3SAT',
 '2019_ARD',
 '2019_ARTE',
 '2019_BFS',
 '2019_DMAX',
 '2019_DW-tv',
 '2019_EinsFestival',
 '2019_HF',
 '2019_K1',
 '2019_K1DOKU',
 '2019_KKA',
 '2019_MDR',
 '2019_N24DOKU',
 '2019_N3_Hamburg',
 '2019_N3_Radio_Bremen',
 '2019_NTV',
 '2019_PHOENIX',
 '2019_PRO7',
 '2019_ProSiebenFun',
 '2019_ProSieben_MAXX',
 '2019_RTL',
 '2019_RTL2',
 '2019_RTLS',
 '2019_RTL_Nitro',
 '2019_RTLplus',
 '2019_SAT1',
 '2019_SWR_BW',
 '2019_SWR_SR',
 '2019_Sat1Gold',
 '2019_Sport1',
 '2019_Tele_5',
 '2019_VOX',
 '2019_WDR',
 '2019_WELT',
 '2019_ZDF',
 '2019_ZDF_Info',
 '2019_ZDFneo',
 '2019_rbb_Berlin',
 '2019_sixx',
 '2019_tagesschau24']

In [20]:
main_station=stations['2019_3SAT']
scrapped_station=de_stations['2019_3SAT']

In [22]:
main_station.find({}).count()

<ipython-input-22-743aad3d2ec3>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  main_station.find({}).count()


12532

In [29]:
scrapped_station.find({}).count()

<ipython-input-29-88099302f91b>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  scrapped_station.find({}).count()


6380

In [42]:
for station_name in station_names:
    main_station=stations[station_name]
    scrapped_station=de_stations[station_name]
    print(f" total stations {main_station.find({}).count()} importable {scrapped_station.find({}).count()} in {station_name}")
    imported=0
    print
    for index, show in enumerate( scrapped_station.find({})) :
        del show['_id']
        found_show = main_station.find_one_and_update(
            {"show_id" : show['show_id']},
            {
                "$set":show
            }
        )
        if found_show:
            imported+=1
    print(f"imported {imported}")

<ipython-input-42-6021072c29fa>:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(f" total stations {main_station.find({}).count()} importable {scrapped_station.find({}).count()} in {station_name}")


 total stations 12532 importable 6380 in 2019_3SAT
imported 6380
 total stations 13280 importable 10929 in 2019_ARD
imported 10929
 total stations 11870 importable 5710 in 2019_ARTE
imported 5710
 total stations 13371 importable 12728 in 2019_BFS
imported 12728
 total stations 11118 importable 1979 in 2019_DMAX
imported 1979
 total stations 24774 importable 3816 in 2019_DW-tv
imported 3816
 total stations 11122 importable 6046 in 2019_EinsFestival
imported 6046
 total stations 13431 importable 6233 in 2019_HF
imported 6233
 total stations 9142 importable 2989 in 2019_K1
imported 2989
 total stations 11842 importable 1715 in 2019_K1DOKU
imported 1715
 total stations 23298 importable 13347 in 2019_KKA
imported 13347
 total stations 15514 importable 5988 in 2019_MDR
imported 5988
 total stations 10481 importable 853 in 2019_N24DOKU
imported 853
 total stations 13667 importable 5462 in 2019_N3_Hamburg
imported 5462
 total stations 14520 importable 5465 in 2019_N3_Radio_Bremen
imported 5465

{'_id': ObjectId('600ea27dc8e8fd3a7ca50c70'), 'show_id': ';1020463513', 'title': 'Lederstrumpf', 'subtitle': 'Der Wildtöter', 'originaltitel': '', 'sender_genre': 'Abenteuerfilm\nFernsehfilm', 'dmb_haupt_genre': 'TV-Film', 'dmb_neben_genre': 'TV-Abenteuerfilm', 'produktion_land': 'Deutschland / Frankreich', 'produktion_jahr': '1969', 'hinweissender': '', 'titel_ergaenzung': 'Der Wildtöter', 'ausstrahlung_start': '', 'ausstrahlung_sender_start': '2019-01-04T13:00:00', 'ausstrahlung_stop': '2019-01-04T14:30:00', 'ausstrahlung_dauer': '90', 'nettolaenge': '88', 'ausstrahlung_sender': '3SAT', 'produzierender_sender': '', 'dmbInfoset_ProduktionsLaender_ProduktionsLand': 'Deutschland', 'dmbFolgetitel': '', 'dmbStaffel': '', 'dmbFolge': '', 'dmbProduktionsJahr': '1969', 'dmbOriginalFolgetitel': '', 'staff': [], 'produktionsLand': [{'land': 'Deutschland', 'kfz': 'D', 'alpha3': 'DEU'}, {'land': 'Frankreich', 'kfz': 'F', 'alpha3': 'FRA'}], 'crew_united_url': '', 'crew_united_productions': [], 'c